# U-Net

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import xarray.plot as xplt
import cftime

In [2]:
import pysteps

Pysteps configuration file found at: /home/henry/anaconda3/envs/downscaling/lib/python3.9/site-packages/pysteps/pystepsrc



In [3]:
cp_model_rotated_pole = ccrs.RotatedPole(pole_longitude=177.5, pole_latitude=37.5)
platecarree = ccrs.PlateCarree()

In [4]:
cpmdata = xr.open_mfdataset("../../../data/2.2km/rcp85/01/pr/*.nc").rename({"pr": "target_pr"})
cpmdata = cpmdata.loc[dict(ensemble_member=1, time=slice("1980-12-01","1982-11-30"))]
cpmdata = cpmdata.reset_coords()[['target_pr']]
cpmdata

<xarray.Dataset>
Dimensions:         (grid_latitude: 606, grid_longitude: 484, time: 720)
Coordinates:
  * time            (time) object 1980-12-01 12:00:00 ... 1982-11-30 12:00:00
  * grid_latitude   (grid_latitude) float64 -4.683 -4.647 -4.611 ... 8.027 8.063
  * grid_longitude  (grid_longitude) float64 353.9 354.0 354.0 ... 364.3 364.3
Data variables:
    target_pr       (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(360, 606, 484), meta=np.ndarray>
Attributes: (12/15)
    collection:      land-cpm
    contact:         ukcpproject@metoffice.gov.uk
    creation_date:   2019-07-30T06:13:12
    domain:          uk
    frequency:       day
    institution:     Met Office Hadley Centre (MOHC), FitzRoy Road, Exeter, D...
    ...              ...
    resolution:      2.2km
    scenario:        rcp85
    source:          UKCP18 realisation from a set of 12 convection-permittin...
    title:           UKCP18 land projections - 2.2km convection-permitting cl...
    version:         v20190731
    Conventions:     CF-1.5

In [5]:
regridded_gcmdata = xr.open_mfdataset('../../../derived_data/60km/rcp85/01/*/day/*.nc')
regridded_gcmdata = regridded_gcmdata.loc[dict(ensemble_member=1, time=slice("1980-12-01","1982-11-30"))]
regridded_gcmdata = regridded_gcmdata.reset_coords()[['pr', 'psl']]
regridded_gcmdata

<xarray.Dataset>
Dimensions:         (grid_latitude: 606, grid_longitude: 484, time: 720)
Coordinates:
  * time            (time) object 1980-12-01 12:00:00 ... 1982-11-30 12:00:00
  * grid_latitude   (grid_latitude) float64 -4.683 -4.647 -4.611 ... 8.027 8.063
  * grid_longitude  (grid_longitude) float64 353.9 354.0 354.0 ... 364.3 364.3
Data variables:
    pr              (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(720, 606, 484), meta=np.ndarray>
    psl             (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(720, 606, 484), meta=np.ndarray>
Attributes: (12/18)
    collection:      land-gcm
    contact:         ukcpproject@metoffice.gov.uk
    creation_date:   2018-11-03T10:11:47
    description:     Precipitation rate
    domain:          uk
    frequency:       day
    ...              ...
    resolution:      60km
    scenario:        rcp85
    source:          UKCP18 global realisation from a set of 15 perturbed var...
    title:           UKCP18 land projections - 60km global climate model, pre...
    version:         v20181122
    Conventions:     CF-1.7

In [64]:
merged_data = xr.merge([regridded_gcmdata, cpmdata])
# select a small subset of the data for trail purposes
merged_data = merged_data.isel({"grid_latitude": slice(512), "grid_longitude": slice(512)})

# split training/test based on date
training_data = merged_data.sel({"time": slice("1980-12-01", "1981-11-30")})
validation_data = merged_data.sel({"time": slice("1981-12-01", "1982-05-30")})
test_data = merged_data.sel({"time": slice("1982-06-01", "1982-11-30")})

In [65]:
training_data

<xarray.Dataset>
Dimensions:         (grid_latitude: 512, grid_longitude: 484, time: 360)
Coordinates:
  * time            (time) object 1980-12-01 12:00:00 ... 1981-11-30 12:00:00
  * grid_latitude   (grid_latitude) float64 -4.683 -4.647 -4.611 ... 5.84 5.86
  * grid_longitude  (grid_longitude) float64 353.9 354.0 354.0 ... 364.3 364.3
Data variables:
    pr              (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(360, 512, 484), meta=np.ndarray>
    psl             (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(360, 512, 484), meta=np.ndarray>
    target_pr       (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(360, 512, 484), meta=np.ndarray>
Attributes: (12/18)
    collection:      land-gcm
    contact:         ukcpproject@metoffice.gov.uk
    creation_date:   2018-11-03T10:11:47
    description:     Precipitation rate
    domain:          uk
    frequency:       day
    ...              ...
    resolution:      60km
    scenario:        rcp85
    source:          UKCP18 global realisation from a set of 15 perturbed var...
    title:           UKCP18 land projections - 60km global climate model, pre...
    version:         v20181122
    Conventions:     CF-1.7

In [66]:
validation_data

<xarray.Dataset>
Dimensions:         (grid_latitude: 512, grid_longitude: 484, time: 180)
Coordinates:
  * time            (time) object 1981-12-01 12:00:00 ... 1982-05-30 12:00:00
  * grid_latitude   (grid_latitude) float64 -4.683 -4.647 -4.611 ... 5.84 5.86
  * grid_longitude  (grid_longitude) float64 353.9 354.0 354.0 ... 364.3 364.3
Data variables:
    pr              (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(180, 512, 484), meta=np.ndarray>
    psl             (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(180, 512, 484), meta=np.ndarray>
    target_pr       (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(180, 512, 484), meta=np.ndarray>
Attributes: (12/18)
    collection:      land-gcm
    contact:         ukcpproject@metoffice.gov.uk
    creation_date:   2018-11-03T10:11:47
    description:     Precipitation rate
    domain:          uk
    frequency:       day
    ...              ...
    resolution:      60km
    scenario:        rcp85
    source:          UKCP18 global realisation from a set of 15 perturbed var...
    title:           UKCP18 land projections - 60km global climate model, pre...
    version:         v20181122
    Conventions:     CF-1.7

In [67]:
test_data

<xarray.Dataset>
Dimensions:         (grid_latitude: 512, grid_longitude: 484, time: 180)
Coordinates:
  * time            (time) object 1982-06-01 12:00:00 ... 1982-11-30 12:00:00
  * grid_latitude   (grid_latitude) float64 -4.683 -4.647 -4.611 ... 5.84 5.86
  * grid_longitude  (grid_longitude) float64 353.9 354.0 354.0 ... 364.3 364.3
Data variables:
    pr              (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(180, 512, 484), meta=np.ndarray>
    psl             (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(180, 512, 484), meta=np.ndarray>
    target_pr       (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(180, 512, 484), meta=np.ndarray>
Attributes: (12/18)
    collection:      land-gcm
    contact:         ukcpproject@metoffice.gov.uk
    creation_date:   2018-11-03T10:11:47
    description:     Precipitation rate
    domain:          uk
    frequency:       day
    ...              ...
    resolution:      60km
    scenario:        rcp85
    source:          UKCP18 global realisation from a set of 15 perturbed var...
    title:           UKCP18 land projections - 60km global climate model, pre...
    version:         v20181122
    Conventions:     CF-1.7

In [103]:
from unet import UNet

In [104]:
model = UNet(2, 1)
model

UNet(
  (inc): DoubleConv(
    (double_conv): Sequential(
      (0): Conv2d(2, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
  )
  (down1): Down(
    (maxpool_conv): Sequential(
      (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (1): DoubleConv(
        (double_conv): Sequential(
          (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (4): BatchNorm2d(128, eps=1e-05, moment

In [82]:
import torch

In [101]:
img_size = 16
training_tensor = torch.stack((torch.from_numpy(training_data.isel({"grid_latitude": slice(img_size), "grid_longitude": slice(img_size)}).isel({"time": slice(2)}).psl.values),  torch.from_numpy(training_data.isel({"grid_latitude": slice(img_size), "grid_longitude": slice(img_size)}).isel({"time": slice(2)}).pr.values)), dim=1)

In [102]:
model(training_tensor)

tensor([[[[ 0.6202,  0.5706,  0.2214,  0.1205,  0.1891,  0.2663,  0.4527,
            0.5021,  0.4242,  0.3176,  0.3444,  0.2189,  0.1755,  0.6847,
            0.4419,  0.0890],
          [ 0.2414,  0.3094, -0.4780, -0.1083, -0.1397, -0.0291,  0.1862,
            0.2579,  0.1034, -0.1111, -0.2812, -0.4519, -0.0406,  0.3725,
            0.2070, -0.5428],
          [ 0.0156,  0.2379,  0.1201, -0.8489, -0.4812, -0.2933, -0.3216,
           -0.2264, -0.3710, -0.5510, -0.9947, -0.9932, -0.9958, -0.7180,
           -0.3610, -0.9370],
          [-0.1934,  0.0385,  0.2049, -0.2464, -0.4022, -0.0401, -0.1055,
           -0.2441, -0.2387, -0.3251, -0.6251, -0.4471, -0.1314,  0.2599,
            0.4373, -0.7219],
          [ 0.0938,  0.3708, -0.3274, -0.3042, -0.2572, -0.0986,  0.0157,
           -0.1350, -0.2778, -0.5033, -0.5006, -0.2677,  0.1657,  0.6060,
            0.4405, -0.6405],
          [ 0.1727,  0.3765, -0.2777,  0.0020,  0.0111, -0.0769, -0.1615,
           -0.1866, -0.2810, -0.4538